# 4-deg (latitude) x 5-deg (longitude) monthly average gridded datasets over Continental US
## OCO-2: XCO2 and SIF; MODIS: EVI

For each process, produce multiple overlapping datasets computed and pooled for a resulting lat/lon offset of only 0.5-degrees

NOTE: Use `geopy` conda environment

---

In [1]:
import sys
sys.path.insert(0, "../src")

import numpy as np
import pandas as pd 
import xarray as xr

from data_utils import augment_dataset

In [2]:
ds_xco2 = xr.open_dataset("../data/exp_pro/OCO2_Lite_XCO2.nc")
ds_sif = xr.open_dataset("../data/exp_pro/OCO2_Lite_SIF740.nc")
ds_evi = xr.open_dataset("../data/exp_pro/MODIS_EVI_conus.nc", engine="h5netcdf")

In [3]:
ds_evi

<xarray.Dataset>
Dimensions:  (time: 72, lon: 1400, lat: 880)
Coordinates:
  * lat      (lat) float64 61.97 61.92 61.88 61.83 ... 18.17 18.12 18.08 18.02
  * lon      (lon) float64 -130.0 -129.9 -129.9 -129.8 ... -60.12 -60.07 -60.02
  * time     (time) datetime64[ns] 2015-01-01 2015-02-01 ... 2020-12-01
Data variables:
    evi      (time, lon, lat) float32 ...

In [4]:
# Compute augmented datasets
# df_aug_xco2 = augment_dataset(ds_xco2)
# df_aug_sif = augment_dataset(ds_sif)
df_aug_evi = augment_dataset(ds_evi)

ImportError: cartopy is required to download/ access NaturalEarth data

In [5]:
# Merge dataframes
ds_augment_conus = (
    pd.merge(df_aug_xco2, df_aug_sif, on=["lon", "lat", "time"], how="outer")
    .set_index(["lon", "lat", "time"])
    .to_xarray()
)
ds_augment_conus

<xarray.Dataset>
Dimensions:   (lat: 79, lon: 129, time: 75)
Coordinates:
  * lon       (lon) float64 -127.0 -126.5 -126.0 -125.5 ... -64.0 -63.5 -63.0
  * lat       (lat) float64 20.5 21.0 21.5 22.0 22.5 ... 58.0 58.5 59.0 59.5
  * time      (time) datetime64[ns] 2014-09-01 2014-10-01 ... 2020-11-01
Data variables:
    xco2      (lon, lat, time) float32 nan nan nan nan nan ... nan nan nan nan
    xco2_var  (lon, lat, time) float32 nan nan nan nan nan ... nan nan nan nan
    sif       (lon, lat, time) float32 nan nan nan nan nan ... nan nan nan nan
    sif_var   (lon, lat, time) float32 nan nan nan nan nan ... nan nan nan nan

In [6]:
ds_augment_conus.to_netcdf("../data/exp_pro/OCO2_4x5deg_monthly_conus_augmented.nc", format="NETCDF4")
ds_augment_conus.close()